# Intro to UAVSAR

```{admonition} Learning Objectives
- find, visualize, analyze UAVSAR data
```

Intro slide deck: https://uavsar.jpl.nasa.gov/education/what-is-uavsar.html 

## Quick facts

| frequency | resolution (rng x azi m) | swath width (km) |
| - | - | - | 
| L-band | 1.8 | 16 | 

Documentation:
* https://uavsar.jpl.nasa.gov/science/documents.html
* https://asf.alaska.edu/data-sets/sar-data-sets/uavsar/

Data Access:

* [NASA Earthdata Suborbital Search](https://search.earthdata.nasa.gov/portal/suborbital/search?fi=UAVSAR&as[instrument][0]=UAVSAR)
* [ASF Vertex SnowEx Grand Mesa Campaign](https://search.asf.alaska.edu/#/?dataset=UAVSAR&mission=Grand%20Mesa,%20CO&resultsLoaded=true&granule=UA_grmesa_27416_21019-017_21021-005_0006d_s01_L090_01-AMPLITUDE_GRD&zoom=3&center=-92.747866,10.530273&productTypes=AMPLITUDE_GRD)

## API data access

In [ ]:
## Glossary